In [1]:
import os
os.environ['PROJ_LIB'] = '/data/keeling/a/sshu3/anaconda2/share/proj'
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy.ma as ma
from scipy.interpolate import griddata
from pyresample import geometry,image, kd_tree
import mycolorbars


In [6]:
## Scatter plot for showing the correlation btw env factor and wetland CH4 emission.
## Historical
# Get N
nclu=NetCDFFile('hist_01_10.nc','r')
ncvar_nonwtld = nclu.variables['ch4_flux_dry'][:]
ncvar_wtld = nclu.variables['ch4_flux_wet'][:]
ncvar_inund = nclu.variables['ch4_flux_inund'][:]
ncvar_wtld_prod = nclu.variables['ch4_prod_wet'][:]
ncvar_inund_prod = nclu.variables['ch4_prod_inund'][:]

latnc = nclu.variables['lat'][:]
lonnc = nclu.variables['lon'][:]
lonnc = ((lonnc - 180) % 360) - 180
ncvar_wetland = ncvar_wtld + ncvar_inund
ncvar_wetland_prod = ncvar_wtld_prod + ncvar_inund_prod

# Get FW
nclu_fw=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_01_10.nc','r')
fw = nclu_fw.variables['FW'][:]

# Get Environmental factors (Tair and Precip)
nclu_env=NetCDFFile('hist_01_10_env.nc','r')
ncvar_temp = nclu_env.variables['tavg_avg_yr'][:]
ncvar_prec = nclu_env.variables['precip_avg_yr'][:]
#ncvar_wliq = nclu_env.variables['wliq_vol_1m_yr'][:]
ncvar_wliq = nclu_env.variables['sm_1_5_avg_yr'][:]
ncvar_wliq = ncvar_wliq * 100.

## Future scenarios
nclu2=NetCDFFile('rcp45_91_00.nc','r')
ncvar2_nonwtld = nclu2.variables['ch4_flux_dry'][:]
ncvar2_wtld = nclu2.variables['ch4_flux_wet'][:]
ncvar2_inund = nclu2.variables['ch4_flux_inund'][:]

ncvar2_wtld_prod = nclu2.variables['ch4_prod_wet'][:]
ncvar2_inund_prod = nclu2.variables['ch4_prod_inund'][:]

ncvar2_wetland = ncvar2_wtld + ncvar2_inund
ncvar2_wetland_prod = ncvar2_wtld_prod + ncvar2_inund_prod

nclu_fw2=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp45.nc','r')
fw2 = nclu_fw2.variables['FW'][:]

nclu_env2=NetCDFFile('rcp45_91_00_env.nc','r')
ncvar2_temp = nclu_env2.variables['tavg_avg_yr'][:]
ncvar2_prec = nclu_env2.variables['precip_avg_yr'][:]
ncvar2_wliq = nclu_env2.variables['wliq_vol_1m_yr'][:]

nclu3=NetCDFFile('rcp85_91_00.nc','r')
ncvar3_nonwtld = nclu3.variables['ch4_flux_dry'][:]
ncvar3_wtld = nclu3.variables['ch4_flux_wet'][:]
ncvar3_inund = nclu3.variables['ch4_flux_inund'][:]

ncvar3_wtld_prod = nclu3.variables['ch4_prod_wet'][:]
ncvar3_inund_prod = nclu3.variables['ch4_prod_inund'][:]

ncvar3_wetland = ncvar3_wtld + ncvar3_inund
ncvar3_wetland_prod = ncvar3_wtld_prod + ncvar3_inund_prod

nclu_fw3=NetCDFFile('/data/jain1/c/sshu3/SBGC/results/get_us_tot/fw_frac_mean_91_00_rcp85.nc','r')
fw3 = nclu_fw3.variables['FW'][:]

nclu_env3=NetCDFFile('rcp85_91_00_env.nc','r')
ncvar3_temp = nclu_env3.variables['tavg_avg_yr'][:]
ncvar3_prec = nclu_env3.variables['precip_avg_yr'][:]
ncvar3_wliq = nclu_env3.variables['wliq_vol_1m_yr'][:]

# Mask out regions outside the CONUS
ncmk=NetCDFFile('surfdata_05x05_13reg.nc','r')
ncvar_mask = ncmk.variables['REGION_MASK_CRU_NCEP'][:]
#ncvar_mask[ncvar_mask<12] = 0
#ncvar_mask[ncvar_mask>12] = 0
#ncvar_mask[ncvar_mask==12] = 1

#ncvar_net = ncvar_net*ncvar_mask
#ncvar2_net = ncvar2_net*ncvar_mask
#ncvar3_net = ncvar3_net*ncvar_mask
ncvar_nonwtld[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_nonwtld[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_wetland[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar_wetland[ncvar_mask<12] = ma.masked  #ncvar3_net*ncvar_mask

ncvar2_nonwtld[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_nonwtld[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_wetland[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_wetland[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask

ncvar3_nonwtld[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar3_nonwtld[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar3_wetland[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_wetland[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask

ncvar_temp[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_temp[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_temp[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_temp[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_temp[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_temp[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask

ncvar_prec[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_prec[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_prec[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_prec[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_prec[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_prec[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask

fw[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
fw[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
fw2[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
fw2[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
fw3[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
fw3[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask

ncvar_wliq[ncvar_mask<12] = ma.masked  #ncvar_net*ncvar_mask
ncvar_wliq[ncvar_mask>12] = ma.masked  #ncvar_net*ncvar_mask
ncvar2_wliq[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar2_wliq[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_wliq[ncvar_mask<12] = ma.masked  #ncvar2_net*ncvar_mask
ncvar3_wliq[ncvar_mask>12] = ma.masked  #ncvar2_net*ncvar_mask


In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar_wetland, ncvar_temp, c=fw, vmin=0, vmax=0.4, cmap=plt.cm.Blues)
plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar_wetland, ncvar_prec, c=fw, vmin=0, vmax=0.4, cmap=plt.cm.Blues)
plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar_wetland, fw, c=ncvar_prec, vmin=0, vmax=2000, cmap=plt.cm.Blues)
plt.colorbar()
#plt.show()
plt.savefig('figs5b.png')

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar_wetland, fw, c=ncvar_temp, vmin=270, vmax=300, cmap=plt.cm.Reds)
plt.colorbar()
#plt.show()
plt.savefig('figs5a.png')

In [8]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar_nonwtld[fw<0.02], ncvar_wliq[fw<0.02], c=ncvar_temp[fw<0.02], vmin=270, vmax=300, cmap=plt.cm.Reds)
plt.xlim([-3,3])
plt.axvline(x=0.0)
plt.colorbar()
#plt.show()
plt.savefig('figs6.png')

In [ ]:
# The following cells are for analyzing the RCP4.5 projection.
diff_wetland = ncvar2_wetland - ncvar_wetland
diff_nonwtld = ncvar2_nonwtld - ncvar_nonwtld
diff_wetland_prod = ncvar2_wetland_prod - ncvar_wetland_prod
diff_fw = fw2 - fw
diff_temp = ncvar2_temp - ncvar_temp
diff_prec = ncvar2_prec - ncvar_prec
diff_wliq = ncvar2_wliq - ncvar_wliq

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland, diff_fw, c=diff_temp, vmin=1, vmax=2.5, cmap=plt.cm.Reds)
plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland[abs(ncvar2_wetland)>0.3], diff_temp[abs(ncvar2_wetland)>0.3])  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland, diff_prec)  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_nonwtld, diff_wliq)  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# The following cells are for analyzing the RCP8.5 projection.
diff_wetland = ncvar3_wetland - ncvar_wetland
diff_nonwtld = ncvar3_nonwtld - ncvar_nonwtld
diff_wetland_prod = ncvar3_wetland_prod - ncvar_wetland_prod
diff_fw = fw3 - fw
diff_temp = ncvar3_temp - ncvar_temp
diff_prec = ncvar3_prec - ncvar_prec
diff_wliq = ncvar3_wliq - ncvar_wliq

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland, diff_fw, c=diff_temp, vmin=3, vmax=6, cmap=plt.cm.Reds)
plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland, diff_wetland_prod)  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_wetland, diff_prec)  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(diff_nonwtld, diff_wliq)  #, c=diff_temp, vmin=-0.1, vmax=0.1, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
# Prepare scatter plot
plt.rcParams.update({'font.size': 16})
plt.scatter(ncvar3_nonwtld, ncvar3_wliq, c=ncvar3_temp, vmin=260, vmax=300, cmap=plt.cm.Reds)
#plt.colorbar()
plt.show()

In [ ]:
#http://matplotlib.org/basemap/users/mapsetup.html
# VWC of wetland & non-wetland

# My personal colormap
my_cmap = mycolorbars.color_ch4diff()

fig = plt.figure(figsize=(15,25))

ax1 = fig.add_subplot(311)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) ",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs1 = map.pcolormesh(x,y,ncvar_wliq[:,:],
#              norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=-1.0, vmax=100),cmap=my_cmap) # use log color scale,                   
              norm=colors.Normalize(vmin=0.1, vmax=0.5),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs1.cmap.set_under('w')
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=18) 

ax1 = fig.add_subplot(312)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP4.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs = map.pcolormesh(x,y,ncvar2_wliq[:,:],
              norm=colors.Normalize(vmin=0.1, vmax=0.5),cmap=plt.cm.jet) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs.cmap.set_under('w')
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=18) 

ax1 = fig.add_subplot(313)
#ax1.set_title("Net $CH_4$ flux (gC-$CH_4$/$m^2$/yr) RCP8.5",fontsize=20)

map = Basemap(llcrnrlon=-119,llcrnrlat=23,urcrnrlon=-63,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)

# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
map.drawstates()
map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
cs = map.pcolormesh(x,y,ncvar3_wliq[:,:],
              norm=colors.Normalize(vmin=0.1, vmax=0.5),cmap=plt.cm.jet) # use log color scale,
#              norm=colors.LogNorm(vmin=0.01, vmax=100),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#cs.cmap.set_under('w')
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=18) 

plt.show()
#plt.savefig('figS6.jpg',dpi=300)